# CHOMP v2
__Misc. Utilities__

__by Sean Gilleran__  
__Last updated November 30__, __2021__  
[https://github.com/seangilleran/chomp2](https://github.com/seangilleran/chomp2)

## Check Language

In [ ]:
import os

import en_core_web_sm
from spacy.language import Language
from spacy_langdetect import LanguageDetector

path = "./corpus"


@Language.factory("language_detector")
def language_detector(nlp, name):
    return LanguageDetector()

nlp = en_core_web_sm.load()
nlp.add_pipe("language_detector", last=True)

for file in [f for f in os.listdir(path) if f.endswith(f".txt")]:

    with open(os.path.join(path, file), "r", encoding="utf-8") as f:
        text = f.read()

    lang = nlp(text)._.language

    if lang["language"] != "en" or lang["score"] < 0.7:
        print(f"{lang}: {file}")

    # Write results to CSV
    with open("lang_check.csv", "a", encoding="utf-8") as f:
        f.write(f"{lang['language']},{lang['score']},{file}\n")

## Convert PDF to TXT

In [ ]:
# TODO